In [1]:
import os
import requests
import urllib
import time
import glob
import json
import urllib.parse
import re
import unicodedata
import json
import datetime
# 構文解析
import CaboCha
# スクレピング
from bs4 import BeautifulSoup
# 機械学習
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### パラメーターの設定

In [2]:
# 保存先ディレクトリ
save_directory = f"./postd"

In [3]:
def write_file(url, file_path):
    r = requests.get(url, stream=True)
    
    if r.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(r.content)
    else:
        print(f"HTTP STATUS ERROR {r.status_code}")

### 記事一覧ページから各ページをテキストに保存する

In [ ]:
'''
for i in range(69, -1, -1):
    if i == 0:
        page_url = f"https://postd.cc"
    else:
        page_url = f"https://postd.cc/page/{i}/"
    # 負荷をかけすぎないように待機
    time.sleep(1)
    # 一覧ページへのリクエスト
    rlist = requests.get(page_url)
    soup = BeautifulSoup(rlist.text, features="html.parser")
    section = soup.find_all("section", attrs={"id":"block-container"})
    for block in section:
        ablocks = block.find_all("a", attrs={"class": "a-transition", "rel": "bookmark"})
        for ablock in ablocks:
            # 負荷をかけすぎないように待機
            time.sleep(1)
            blog_name = os.path.basename(os.path.dirname(ablock['href']))
            blog_url = f"https://postd.cc/{blog_name}"

            file_name = "{}.html".format(blog_name)
            html_path = os.path.join(save_directory, file_name)
            write_file(url=blog_url, file_path=html_path)
            print(blog_url)
'''

### MySQLクラス

In [4]:
import MySQLdb

class MysqlClass:
    connection = None
    cursor = None
    
    def connect(self):
        # データベースへの接続
        self.connection = MySQLdb.connect(
        user='root',
        passwd='',
        host='192.168.1.44',
        db='postd',
        charset='utf8')
        # カーソルの作成
        self.cursor = self.connection.cursor()

    def disconnect(self):
        # 接続を閉じる
        self.connection.close()

    def add_article (self, entry_title, post_date, tag, content, meta_info):
        self.connect()
        sql = 'INSERT INTO article (entry_title, post_date, tag, content, meta_info) VALUES (%s, %s, %s, %s, %s)'
        self.cursor.execute(sql, (entry_title, post_date, tag, content, meta_info))
        self.connection.commit()
        self.disconnect()
        
    def set_article_by_id(self, column, id, value):
        self.connect()
        
        sql = 'UPDATE article SET {0} = %s where id = %s'.format(column)
        self.cursor.execute(sql, (json.dumps(value), id))
        self.connection.commit()
        self.disconnect()
        
    def find_all(self, columns='*'):
        self.connect()
        if columns == '*':
            sql = 'SELECT {0} FROM article ORDER BY post_date DESC'.format(columns)
        else:
            sql = 'SELECT {0} FROM article ORDER BY post_date DESC'.format(','.join(columns))
        self.cursor.execute(sql)
        rows = self.cursor.fetchall()
        self.disconnect()  
        return rows

    def find_ids(self, limit, offset=0):
        self.connect()
        sql = 'SELECT id FROM article ORDER BY post_date DESC LIMIT %s OFFSET %s'
        self.cursor.execute(sql, (limit, offset))
        rows = self.cursor.fetchall()
        self.disconnect()  
        return rows

    def find_article_by_id(self, columns, id ):
        self.connect()
        sql = 'SELECT {0} FROM article WHERE id = %s'.format(','.join(columns))
        self.cursor.execute(sql, (id,))
        row = self.cursor.fetchone()
        self.disconnect()  
        return row

    def find_article_by_entry_title(self, columns, entry_title):
        self.connect()
        sql = 'SELECT {0} FROM article WHERE entry_title LIKE %s'.format(','.join(columns))
        self.cursor.execute(sql, ('%' + entry_title + '%',))
        rows = self.cursor.fetchone()
        self.disconnect()
        return rows

mysql_class = MysqlClass()

### 記事ファイルから抽出処理

In [5]:
#　全角文字に変換
translation_table = str.maketrans(dict(zip('()!', '（）！')))

def cleanse(text):
    # Normalization Form Compatibility Composition
    text = unicodedata.normalize('NFKC', text).translate(translation_table)
    # スペースを半角スペースに統一
    text = re.sub(r'\s+', ' ', text)
    return text

def scrape(soup):
    
    # 見出しの抽出
    entry_title = soup.find('div', attrs={'class': 'block-titles'}).find('h1', attrs={'class': 'block-entry-title'}).get_text()
        
    # 日付の抽出
    post_date = soup.find('div', attrs={'class': 'block-titles'}).find('div', attrs={'class': 'block-date'}).get_text()
    post_date = datetime.datetime.strptime(post_date, '%Y年%m月%d日').strftime('%Y-%m-%d')
    
    # カテゴリの抽出
    tag_block = soup.find('div', attrs={'class': 'tagcloud', 'itemprop': 'keywords'})
    if(tag_block != None):
        tag = ','.join(cleanse(taga.text.strip())
                      for taga in tag_block.find_all(['a'])
                      if len(taga.text.strip()) > 0)
    else:
        tag = None
        
    # 本文の抽出
    honbun = soup.find('div', attrs={'class': 'block-text-content'}).find('div', attrs={'class': 'markdown-content'})

    # __EOS__ の挿入
    for block in honbun.find_all(['br', 'p', 'h1', 'h2', 'h3', 'h4', 'li', 'strong']):
        if len(block.text.strip()) > 0 and block.text.strip()[-1] not in ['。', '！']:
            block.append('<__EOS__>')
    # 本文の抽出
    text = '\n'.join([cleanse(block.text.strip())
                      for block in honbun.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'li'])
                      if len(block.text.strip()) > 0])
    # タイトルの抽出
    title = cleanse(soup.title.text.replace(' | POSTD', ''))
    return entry_title, post_date, tag, text, title


### htmlデータをDBに登録

In [6]:
for filename in glob.glob('./postd/*.html'):
    soup = BeautifulSoup(open(filename, encoding="utf-8"), features="html.parser")
    entry_title, post_date, tag, content, title = scrape(soup)
    url = 'https://postd.cc/{0}'.format(urllib.parse.quote(os.path.basename(filename).replace('.html', '/')))
    meta_info = json.dumps({'url': url, 'title': title})
    try:
        mysql_class.add_article(entry_title, post_date, tag, content, meta_info)
    except:
        import traceback
        traceback.print_exc()
        mysql_class.disconnect()


### 単一実行用

In [7]:
filename = './postd/q-and-a.html'
soup = BeautifulSoup(open(filename, encoding="utf-8"), features="html.parser")
entry_title, post_date, tag, content, title = scrape(soup)
url = 'https://postd.cc/{0}'.format(urllib.parse.quote(os.path.basename(filename).replace('.html', '/')))
meta_info = json.dumps({'url': url, 'title': title})
print(content)

近い将来、AIが人間社会に与える影響とは?<__EOS__> 近い将来、いくつかの大きなイノベーションが起こる可能性は非常に高いでしょう。自動運転車は既に本格的な開発および試験段階にあり、近いうちに販売を開始すると公約した自動車メーカーもあります（一方で困難を認識し、より慎重になっているメーカーもあります）。また、コンピュータビジョンや脚式移動の改善により、農業やその他の事業の基盤作り、あるいは（特に高齢者や病人に対する）家の雑用サポートなどを含む、非構造化環境向けのロボットも実用的になっています。そして、機械が言語の理解を向上させるにつれて、検索エンジンや携帯電話の “パーソナルアシスタント” は、その力点をWebページのインデクス作成からWebページの理解へとシフトさせており、それが質問の回答や新しい情報の統合、あるいはアドバイスの提供や複数情報からの結論の導出というような性能の質的な向上につながっています。AIはまた、情報の量と複雑さが人間の能力にとって壁となっていた、システム生物学のような科学分野にも大きな影響を与えるかもしれません。 一般的な誤解 ロボットが “支配する力” を握ろうとしている。 AIのシステムが人間よりも知能が高くなるのはいつか?をご覧ください。AIの進歩の大部分はコンピュータとロボットをより有用にするためのものであり、その歩みは段階的です。ただし、人間による制御を維持するという問題は、長期的には重要になるでしょう。
近い将来、いくつかの大きなイノベーションが起こる可能性は非常に高いでしょう。自動運転車は既に本格的な開発および試験段階にあり、近いうちに販売を開始すると公約した自動車メーカーもあります（一方で困難を認識し、より慎重になっているメーカーもあります）。また、コンピュータビジョンや脚式移動の改善により、農業やその他の事業の基盤作り、あるいは（特に高齢者や病人に対する）家の雑用サポートなどを含む、非構造化環境向けのロボットも実用的になっています。そして、機械が言語の理解を向上させるにつれて、検索エンジンや携帯電話の “パーソナルアシスタント” は、その力点をWebページのインデクス作成からWebページの理解へとシフトさせており、それが質問の回答や新しい情報の統合、あるいはアドバイスの提供や複数情報からの結論の導出というような性能の質

### 構文解析 係り受け解析

In [8]:
cabocha = CaboCha.Parser('-n1')
# 「。」「！」「__EOS__」を文の末尾みなすための正規表現オブジェクトの作成
ptn_sentence = re.compile(r'(^|。|！|<__EOS__>)\s*(.+?)(?=(。|！|<__EOS__>))', re.M)

# 文章の分割を行う
def split_into_sentences(text):
    sentences = []
    for m in ptn_sentence.finditer(text):
        sentences.append((m.group(2), m.start(2)))
    return sentences

def parse_sentence(sentence_str, sentence_begin, chunks, tokens):
    # 係り受けきの取得
    tree = cabocha.parse(sentence_str)

    offset = sentence_begin
    chunk_id_offset = len(chunks)
    text = sentence_str
    
    # chunkサイズごとにループ
    for i in range(tree.chunk_size()):
        chunk = tree.chunk(i)
        chunk_begin = None
        # tokenごとにループ
        for j in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
            token = tree.token(j)
            features = token.feature.split(',')
            token_begin = text.find(token.surface) + offset
            token_end = token_begin + len(token.surface)
            if chunk_begin is None:
                chunk_begin = token_begin

            tokens.append({
                'begin': token_begin,
                'end':   token_end,
                'lemma': features[-3], # 単語の原型
                'POS':   features[0], # 単語の品詞
                'POS2':  features[1], # 品詞の詳細
                'NE':    token.ne, # 固有表現か（0は固有表現ではない）
            })

            text = text[token_end-offset:]
            offset = token_end

        chunk_end = token_end
        if chunk.link == -1:
            link = -1
        else:
            # チャンクのIDを手前に出現したチャンス数だけずらす
            link = chunk.link + chunk_id_offset
        chunks.append({
            'begin':    chunk_begin,
            'end':      chunk_end,
            'link':     ('chunk', link),
        })

def parse(text):
    sentences = []
    chunks = []
    tokens = []
    sentence_begin = 0
    
    for sentence_str, sentence_begin in split_into_sentences(text):
        parse_sentence(sentence_str, sentence_begin, chunks, tokens)
        sentence_end = chunks[-1]['end']

        sentences.append({
            'begin':    sentence_begin,
            'end':      sentence_end,
        })

    return sentences, chunks, tokens


### sentense, chunk, tokenの登録を行う

In [9]:
columns = ['id', 'content']
try:
    all_data = mysql_class.find_all(['id', 'content'])
    for data in all_data:
        sentence, chunks, tokens = parse(data[columns.index('content')])
        mysql_class.set_article_by_id('sentence', data[columns.index('id')], sentence)
        mysql_class.set_article_by_id('chunk', data[columns.index('id')], chunks)
        mysql_class.set_article_by_id('token', data[columns.index('id')], tokens)
except:
    import traceback
    traceback.print_exc()
    mysql_class.disconnect()

### sentense, chunk, tokenの参照

In [11]:
for id_row in mysql_class.find_ids(limit=1):
        columns = ['content', 'token', 'chunk', 'sentence']
        row = mysql_class.find_article_by_id(columns, id_row[0])
        content = row[columns.index('content')]
        tokens = json.loads(row[columns.index('token')])
        chunks = json.loads(row[columns.index('chunk')])
        sentences = json.loads(row[columns.index('sentence')])

        print('tokens:')
        for token in tokens:
            print('  ', token['POS'], '\t', content[token['begin']:token['end']])

        print('chunks:')
        for chunk in chunks:
            _, link = chunk['link']
            print('  ', content[chunk['begin']:chunk['end']])
            if link != -1:
                parent = chunks[link]
                print('\t-->', content[parent['begin']:parent['end']])
            else:
                print('\t-->', 'None')

        print('sentences:')
        for sentence in sentences:
            print('  ', content[sentence['begin']:sentence['end']])


tokens:
   名詞 	 Gutenberg
   助詞 	 と
   名詞 	 WordPress
   助詞 	 に関する
   名詞 	 騒動
   助詞 	 は
   記号 	 、
   名詞 	 PHP
   助詞 	 の
   名詞 	 終焉
   助詞 	 に
   動詞 	 つながる
   名詞 	 最新
   名詞 	 記事
   助動詞 	 です
   名詞 	 深呼吸
   助詞 	 を
   動詞 	 し
   助詞 	 て
   動詞 	 ください
   記号 	 、
   名詞 	 みなさん
   名詞 	 トロール
   助詞 	 は
   名詞 	 無視
   動詞 	 し
   記号 	 、
   名詞 	 Mark
   名詞 	 Twain
   助詞 	 と
   名詞 	 Fidel
   名詞 	 Castro
   助詞 	 と
   名詞 	 PHP
   助詞 	 と
   助詞 	 の
   名詞 	 共通
   名詞 	 点
   助詞 	 を
   動詞 	 見
   助詞 	 て
   動詞 	 いき
   助動詞 	 ましょ
   助動詞 	 う
   接続詞 	 そして
   記号 	 、
   副詞 	 もっと
   名詞 	 重要
   助動詞 	 な
   名詞 	 の
   助詞 	 は
   記号 	 、
   名詞 	 スタート
   名詞 	 アップ
   助詞 	 や
   名詞 	 スモール
   名詞 	 ビジネス
   助詞 	 にとって
   記号 	 、
   名詞 	 PHP
   助詞 	 が
   名詞 	 今
   助詞 	 でも
   名詞 	 合理
   名詞 	 的
   助動詞 	 な
   名詞 	 選択
   助動詞 	 で
   助動詞 	 ある
   名詞 	 理由
   助動詞 	 です
   名詞 	 PHP
   助詞 	 は
   動詞 	 い
   助動詞 	 つ
   助詞 	 から
   動詞 	 廃れ
   動詞 	 始め
   助動詞 	 た
   名詞 	 の
   助詞 	 か
   記号 	 “
   名詞 	 PHP
   助詞 	 は
   副詞 	 もう
   名詞 	 ダメ
   助動詞 	 だ
   記号 	 ”
 

   動詞 	 せる
   名詞 	 こと
   助詞 	 を
   動詞 	 決め
   助動詞 	 た
   名詞 	 プロジェクト
   助詞 	 に対して
   記号 	 、
   形容詞 	 新しい
   名詞 	 開発
   名詞 	 者
   助詞 	 が
   動詞 	 見つかる
   名詞 	 こと
   助詞 	 を
   名詞 	 お祈り
   動詞 	 致し
   助動詞 	 ます
   名詞 	 開発
   助詞 	 の
   名詞 	 コスト
   助詞 	 は
   動詞 	 抑え
   動詞 	 られ
   助動詞 	 ます
   接続詞 	 というのも
   記号 	 、
   名詞 	 採用
   名詞 	 市場
   助詞 	 に
   動詞 	 出
   助詞 	 て
   動詞 	 いる
   名詞 	 開発
   名詞 	 者
   助詞 	 の
   名詞 	 能力
   助詞 	 は
   形容詞 	 高く
   記号 	 、
   接続詞 	 また
   名詞 	 一
   助詞 	 から
   動詞 	 作り直す
   名詞 	 必要
   助詞 	 は
   動詞 	 あり
   助動詞 	 ませ
   助動詞 	 ん
   名詞 	 利用
   動詞 	 できる
   名詞 	 オープン
   名詞 	 ソース
   助詞 	 の
   名詞 	 プロジェクト
   助詞 	 は
   名詞 	 たくさん
   名詞 	 存在
   動詞 	 し
   助詞 	 て
   動詞 	 い
   助動詞 	 ます
   名詞 	 開発
   助詞 	 の
   名詞 	 コスト
   助詞 	 は
   動詞 	 抑え
   動詞 	 られ
   助動詞 	 ます
   接続詞 	 というのも
   記号 	 、
   名詞 	 採用
   名詞 	 市場
   助詞 	 に
   動詞 	 出
   助詞 	 て
   動詞 	 いる
   名詞 	 開発
   名詞 	 者
   助詞 	 の
   名詞 	 能力
   助詞 	 は
   形容詞 	 高く
   記号 	 、
   接続詞 	 また
   名詞 	 一
   助詞 	 から
   動詞 	 作り直す
   名詞 	 必要
   助詞 	 は


	--> 対処できます
   対処できます
	--> None
   Reactや
	--> Vue、
   Vue、
	--> Abgularのような
   Abgularのような
	--> 最新の
   最新の
	--> フロントエンドの
   フロントエンドの
	--> フレームワークと
   フレームワークと
	--> 適合する
   うまく
	--> 適合する
   適合する
	--> APIを
   APIを
	--> 構築するのに
   構築するのに
	--> 言語です
   PHPは
	--> 言語です
   優れた
	--> 言語です
   言語です
	--> None
   PHPの
	--> ビジネス感覚
   ビジネス感覚
	--> None
   PHPが
	--> 使用されている
   Webサイトの
	--> 80パーセントで
   80パーセントで
	--> 使用されている
   使用されている
	--> 話に
   話に
	--> 戻ります
   戻ります
	--> None
   経済力に
	--> つなげてください
   つなげてください
	--> None
   つまり、
	--> 存在しているのですから
   PHPの
	--> 開発者は
   開発者は
	--> 存在しているのですから
   山ほど
	--> 存在しているのですから
   存在しているのですから
	--> None
   あなたにとっての
	--> 利点は
   利点は
	--> ものです
   以下のような
	--> ものです
   ものです
	--> None
   開発者を
	--> 雇うのは、
   雇うのは、
	--> 簡単です
   極めて
	--> 簡単です
   簡単です
	--> None
   不明瞭な
	--> フレームワークで
   フレームワークで
	--> 間に合わせる
   間に合わせる
	--> ことを
   ことを
	--> 決めた
   決めた
	--> プロジェクトに対して、
   プロジェクトに対して、
	--> 見つかる
   新しい
	--> 開発者が
   開発者が
	--> 見つかる
   見つかる
	--> ことを
   ことを
	--> お祈り致します
   お祈り致します
	--> None
   開発

In [12]:
columns = ['id', 'entry_title', 'post_date', 'tag', 'content', 'meta_info', 'sentence', 'token', 'similarity']

### TF-IDFを計算する

In [20]:
data = []
ids = []

find_all_data = mysql_class.find_all(columns)
for row in find_all_data:
    # 文書ごとの単語の原型から空白区切りでdataに格納
    data.append(' '.join([token['lemma'] for token in json.loads(row[columns.index('token')])]))
    ids.append(row[columns.index('id')])

# 文書集合を元にして各文書中の単語のTF-IDF値を計算
vectorizer = TfidfVectorizer(analyzer='word', max_df=0.5)
vecs = vectorizer.fit_transform(data)

index = 0
for id, vec in zip(ids, vecs.toarray()):
    meta_info = json.loads(find_all_data[index][columns.index('meta_info')])
    title = meta_info['title']
    print(id , title)
    index += 1
    
    for w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1], reverse=True)[:10]:
        lemma = vectorizer.get_feature_names()[w_id]
        print('\t{0:s}: {1:f}'.format(lemma, tfidf))

54 PHPはもうダメだ、PHP万歳！
	ビジネス: 0.252688
	上級: 0.234473
	サイト: 0.188685
	雇う: 0.181705
	お祈り: 0.149113
	ヒゲ: 0.149113
	スモール: 0.129636
	廃れる: 0.129636
	致す: 0.129636
	終焉: 0.125761
89 70%の人がフェイクニュースに懸念 — Google の闘い
	報道: 0.289511
	機関: 0.203878
	ニュース: 0.190931
	ロシア: 0.185237
	マレーシア: 0.180408
	ブック: 0.176394
	ウクライナ: 0.169980
	フェイス: 0.167452
	事件: 0.144755
	オランダ: 0.141726
178 データサイエンティストにおいて、最も需要のあるスキルとは
	サイエンティスト: 0.602251
	求人: 0.355333
	スキル: 0.288436
	需要: 0.190947
	サイエンス: 0.180826
	機械: 0.158557
	学習: 0.144955
	おすすめ: 0.131951
	情報: 0.122141
	雇用: 0.118444
179 JavaScriptとオブジェクト指向プログラミング
	オブジェクト: 0.598996
	クラス: 0.438188
	関数: 0.181245
	カプセル: 0.175637
	指向: 0.156177
	継承: 0.156100
	アクセス: 0.135809
	インスタンス: 0.132433
	プログラミング: 0.107488
	ユース: 0.106957
424 気をつけよう:プログラミングのキャリアの話 – 後編
	デスク: 0.213184
	ソート: 0.194548
	プラットフォーム: 0.187185
	トップ: 0.177943
	アルゴリズム: 0.174199
	アプリケーション: 0.170327
	言語: 0.157562
	プログラミング: 0.140883
	習得: 0.137088
	すべて: 0.132669
273 気をつけよう:プログラミングのキャリアの話 – 前編
	エンド: 0.407963
	バック: 0.299024
	ソケット: 0.2957

	レイアウト: 0.614234
	グリッド: 0.218013
	ページ: 0.214907
	属性: 0.179925
	組み: 0.174921
	組む: 0.171016
	指定: 0.142199
	コンテンツ: 0.137763
	外観: 0.114703
	マージン: 0.108860
683 Dockerコンテナが遅くなるもう一つの原因
	カーネル: 0.475445
	コンテナ: 0.272544
	ベア: 0.189527
	メタル: 0.181260
	プロセス: 0.173475
	稼働: 0.167945
	デバッグ: 0.148838
	パフォーマンス: 0.147079
	リソース: 0.137919
	呼び出し: 0.130210
731 カオステストでHTTP/2の問題を見つけ出す
	パケット: 0.400904
	損失: 0.305765
	カオス: 0.301915
	サイドカー: 0.252965
	遅延: 0.233477
	不利益: 0.201276
	ストリーム: 0.193716
	サービス: 0.186396
	リクエスト: 0.179261
	テスト: 0.166589
124 15年目のVim
	プラグ: 0.464297
	イン: 0.341005
	バッファ: 0.236910
	ファイル: 0.235152
	ターミナル: 0.205765
	コマンド: 0.194731
	ペイン: 0.169594
	検索: 0.166107
	開く: 0.160687
	ファジー: 0.147360
595 ディープラーニングの限界
	幾何: 0.348731
	モデル: 0.290725
	人間: 0.278363
	学習: 0.276319
	空間: 0.247968
	訓練: 0.243556
	マッピング: 0.195002
	入力: 0.181295
	変換: 0.166823
	勾配: 0.137628
752 PythonとKerasを使ってAlphaZero AIを自作する
	ゲーム: 0.379473
	プレイヤー: 0.377310
	対戦: 0.317063
	バリュー: 0.203826
	ファイル: 0.195435
	対局: 0.178054
	アルゴリズム: 0.174751
	学習: 0

	ディスク: 0.300073
	読み込み: 0.295507
	直接: 0.231095
	パフォーマンス: 0.183929
	ディレクトリ: 0.143916
	テスト: 0.141796
	オプション: 0.136549
110 Swype式キーボードを改良する
	距離: 0.611984
	文字: 0.355375
	単語: 0.291006
	キーボード: 0.238495
	平均: 0.184159
	当たり: 0.178830
	英語: 0.176659
	キー: 0.143988
	親指: 0.143585
	成績: 0.138028
482 CPUマスク
	マスク: 0.642166
	ビット: 0.356589
	マクロ: 0.313587
	パラメータ: 0.233424
	関数: 0.193817
	カーネル: 0.168771
	マップ: 0.141754
	オペランド: 0.131436
	与える: 0.117358
	番号: 0.100160
13 レガシープロトコル向けレガシー検索エンジンの構築
	検索: 0.329962
	ギガバイト: 0.263978
	ファイル: 0.251119
	インターネット: 0.232965
	インデックス: 0.225643
	空間: 0.211242
	プロトコル: 0.208991
	クロール: 0.188560
	エンジン: 0.183920
	ポート: 0.155765
791 JavaScript モジュールの現状
	モジュール: 0.621615
	モード: 0.223185
	ポート: 0.213386
	イン: 0.186954
	スムーズ: 0.139201
	移行: 0.134654
	作者: 0.131437
	挙動: 0.129640
	互換: 0.120177
	エコ: 0.118082
366 IRCは死んだ。IRCよ栄えよ。
	ネットワーク: 0.374210
	ユーザ: 0.372677
	チャネル: 0.281483
	衰退: 0.259961
	成長: 0.220694
	チャット: 0.162862
	庭園: 0.135634
	打撃: 0.134654
	よる: 0.121098
	指摘: 0.099039
781 2017年JavaScriptのテスト概論

	割合: 0.095705
	相対: 0.083357
	人気: 0.075995
	中心: 0.067994
	占める: 0.065183
	パーセント: 0.064138
126 JVMはそんなに重くない
	重い: 0.409351
	インストール: 0.196626
	膨張: 0.130356
	手順: 0.126746
	いじる: 0.121366
	ファイル: 0.120188
	肥大: 0.116063
	そんなに: 0.110264
	製品: 0.109891
	あなた: 0.107757
704 ニューラルネットワークの動物園 : ニューラルネットワーク・アーキテクチャのチートシート（前編）
	ニューロン: 0.440234
	ネットワーク: 0.371005
	入力: 0.343954
	訓練: 0.272231
	原文: 0.213480
	ノード: 0.155583
	セル: 0.131855
	出力: 0.130290
	アーキテクチャ: 0.126399
	学習: 0.125471
51 学校では習わないコーディングスキル:オブジェクト所有権
	所有: 0.732447
	オブジェクト: 0.350423
	解放: 0.260090
	割り当て: 0.210749
	可変: 0.168460
	メモリ: 0.139518
	関数: 0.122961
	スキル: 0.105483
	言語: 0.103841
	規約: 0.102599
259 私たちはいかにして環状線で”悪さをする列車”を捕まえたか
	インシデント: 0.675253
	列車: 0.541128
	環状: 0.213066
	信号: 0.140234
	干渉: 0.130117
	発生: 0.124407
	マレー: 0.096848
	走る: 0.089394
	ブレーキ: 0.084198
	原因: 0.076903
642 一から学ぶベジェ曲線
	線分: 0.762636
	端点: 0.508424
	ならう: 0.227552
	アニメーション: 0.142337
	ござる: 0.133615
	ありがとう: 0.130408
	厳密: 0.123396
	リクエスト: 0.108087
	皆さん: 0.090443
	表現: 0.086810
15 型クラスはインタ

	ファイル: 0.156521
	オープン: 0.152147
	モジュール: 0.151738
396 パスワードリセット・リンクが漏れていませんか?
	リセット: 0.501052
	パスワード: 0.465417
	漏洩: 0.430054
	リンク: 0.260560
	トーク: 0.192012
	ユーザ: 0.126833
	サイト: 0.124934
	クン: 0.112521
	トー: 0.111399
	フォーム: 0.106298
168 もしもディスプレイが壊れたら（ヒマラヤで）
	トップ: 0.381472
	ラップ: 0.369602
	キーボード: 0.283145
	スマ: 0.205007
	接続: 0.180904
	キー: 0.158991
	とき: 0.156266
	ディスプレイ: 0.146048
	画面: 0.142396
	押す: 0.124337
785 畳み込みニューラルネットワークの仕組み
	画像: 0.619617
	特徴: 0.364571
	ピクセル: 0.333617
	畳み込む: 0.244502
	一致: 0.164206
	重み: 0.135134
	結合: 0.114195
	ウィンドウ: 0.105604
	投票: 0.103771
	判定: 0.094017
597 たった1バイトの書き込みが引き起こすルート権限での実行の脆弱性
	確保: 0.362146
	メモリ: 0.305672
	悪用: 0.229778
	攻撃: 0.224592
	バッファ: 0.221542
	領域: 0.218254
	解放: 0.174302
	書き込む: 0.164957
	関数: 0.164808
	フラグ: 0.162200
663 Riotのオンラインサービス運用:第1部
	プレイヤ: 0.394500
	コンテナ: 0.334861
	アプリケーション: 0.247327
	ネットワーク: 0.207977
	チーム: 0.206011
	オーバレイ: 0.200327
	インフラ: 0.153341
	スケジューリング: 0.141348
	サーバ: 0.139477
	環境: 0.116321
583 何でもSSHでやってしまいませんか?
	サーバー: 0.586973
	接続: 0.295054

...

	テスト: 0.523007
	デザイン: 0.356621
	プロダクト: 0.353809
	ユーザ: 0.218345
	アンケート: 0.168803
	ページ: 0.116137
	仮説: 0.115002
	意思: 0.108690
	あなた: 0.108530
	スクロール: 0.107996
7 開発者の生産性:Noと言える技術
	ミーティング: 0.585830
	スケジュール: 0.358025
	製作: 0.239783
	毎日: 0.184898
	午後: 0.163883
	木曜日: 0.142458
	優先: 0.138299
	トレイ: 0.136257
	集中: 0.128489
	こなす: 0.127604
44 Node.jsフロー制御 Part 1 – コールバック地獄 vs. Async vs. Highland
	フロー: 0.383301
	アプローチ: 0.361675
	ステップ: 0.258466
	コール: 0.231319
	ルート: 0.214780
	バック: 0.211828
	渡し: 0.171056
	ライブラリ: 0.162568
	アプリケーション: 0.147612
	エラー: 0.146164
787 クライアントとデザイナのためのUX/UIデザインの手引き
	デザイナ: 0.844479
	デザイン: 0.229736
	プロジェクト: 0.146697
	仕事: 0.142944
	プロダクト: 0.113021
	プロセス: 0.074228
	企業: 0.072570
	信頼: 0.071376
	スタジオ: 0.069388
	コミュニケーション: 0.065015
714 ケーススタディ: アプリ “OneShot” をデザインした1週間
	画像: 0.406094
	ハイライト: 0.315424
	ショット: 0.274396
	切り抜き: 0.252728
	デザイン: 0.250699
	スクリーン: 0.250627
	選択: 0.168079
	ユーザ: 0.140093
	下部: 0.121665
	完成: 0.116060
487 @extendを使うべき時、@mixinを使うべき時
	セレクタ: 0.371817
	宣言: 0.302316
	ルール: 0.2227

	開始: 0.111838
	顧客: 0.110312
	大幅: 0.107737
340 なぜ採用される言語とされない言語があるのか
	言語: 0.501203
	調査: 0.402656
	型付け: 0.186804
	静的: 0.185266
	プロジェクト: 0.155025
	採用: 0.147133
	回答: 0.141194
	答える: 0.140640
	疑問: 0.121706
	解析: 0.111100
202 セレブたちのデータ盗難から学ぶセキュリティ
	パスワード: 0.316398
	人物: 0.299151
	掲示板: 0.226011
	流出: 0.217462
	アカウント: 0.215144
	転売: 0.200108
	画像: 0.194397
	ハッカー: 0.186845
	ターゲット: 0.183844
	認証: 0.156079
657 勝利のためのD言語
	言語: 0.363674
	コンパイル: 0.328244
	コンテキスト: 0.177666
	生成: 0.169004
	レベル: 0.153880
	プログラミング: 0.152030
	効率: 0.124793
	メタ: 0.117058
	アンパック: 0.112185
	表現: 0.103797
639 ポール・グレアムによる「経験することと自覚について」
	経験: 0.295507
	読書: 0.239885
	読む: 0.225904
	体験: 0.210941
	定式: 0.207539
	忘れる: 0.199512
	コンパイル: 0.162438
	覚える: 0.150152
	内容: 0.147999
	考え: 0.140839
45 多くの若きプログラマたちが学ぶべきこと
	クラス: 0.668965
	名前: 0.226222
	インターン: 0.222263
	テクスチャ: 0.207523
	統制: 0.192676
	肥大: 0.146646
	踏み出す: 0.143577
	自己: 0.112206
	やすい: 0.101664
	秩序: 0.100497
350 型付けを活用してテストを減らす:静的型を使ったTDD Part 1
	テスト: 0.748927
	設計: 0.147690
	目標: 0.129656
	クラス: 0.

	アルゴリズム: 0.302016
	蓄える: 0.227059
	能力: 0.218589
	アドホック: 0.217156
	コンテスト: 0.209249
	取り組む: 0.200890
	使いこなせる: 0.197027
	マスター: 0.168421
83 キャリアプランニング: 経験のあるエンジニアはどこへ行ってしまうのか?
	年数: 0.339406
	肩書き: 0.322932
	人材: 0.164252
	キャリア: 0.162391
	まあ: 0.159746
	レーザー: 0.138334
	チームリーダー: 0.132313
	寂しい: 0.132313
	異動: 0.132313
	パス: 0.128244
355 8つのDocker開発パターン
	コンテナ: 0.800865
	依存: 0.166179
	環境: 0.160499
	ボリューム: 0.146372
	共有: 0.124845
	関係: 0.122670
	アプリケーション: 0.105628
	外面: 0.088010
	インストール: 0.079757
	パターン: 0.078515
32 機械学習とNode.jsを使用してInstagramユーザの性別を知る
	訓練: 0.412698
	分類: 0.261541
	データセット: 0.258734
	ネットワーク: 0.235017
	入力: 0.188631
	ベクトル: 0.176781
	女性: 0.169890
	ハッシュ: 0.164046
	性別: 0.163264
	単語: 0.160164
120 専門職の移り変わり（つまりMozilla Labsの閉鎖）
	チーム: 0.238217
	閉鎖: 0.198115
	空間: 0.164714
	グループ: 0.160339
	組織: 0.155054
	彼ら: 0.154575
	関与: 0.153644
	人々: 0.143752
	成功: 0.141576
	学ぶ: 0.139509
73 なぜsystemdなのか?
	サービス: 0.522425
	起動: 0.461999
	プロセス: 0.301739
	システム: 0.176985
	ソケット: 0.157768
	カーネル: 0.145368
	スクリプト: 0.144837
	デーモン:

	見極める: 0.130267
	講じる: 0.129294
	ソフトウェア: 0.121852
609 あなたは全部知っていますか?プログラミングの業界用語30選
	用語: 0.303319
	バグ: 0.293147
	レイヤ: 0.220627
	業界: 0.186431
	カモ: 0.161348
	ジミー: 0.161348
	アヒル: 0.145405
	アニメーション: 0.127680
	おもちゃ: 0.124330
	クイーン: 0.121011
773 分散型メッセージングミドルウェアの詳細比較
	メッセージ: 0.571533
	スループット: 0.304640
	ブローカ: 0.278486
	メッセージキュー: 0.266340
	ブロー: 0.260794
	カード: 0.157717
	送信: 0.151931
	キュー: 0.143008
	受信: 0.131372
	分散: 0.126083
369 Sqwiggleが最初の課金ユーザを獲得した3つの方法
	製品: 0.517170
	弊社: 0.284983
	市場: 0.155383
	ネル: 0.149774
	ファ: 0.145872
	創業: 0.121962
	ドリップ: 0.111548
	取引: 0.109889
	顧客: 0.108292
	日間: 0.101123
551 エドガー・ダイクストラが綴った「”自然言語プログラミング”の愚かしさについて」
	機械: 0.429437
	母国: 0.343221
	記号: 0.273602
	体系: 0.213499
	インターフェース: 0.143478
	無意味: 0.140387
	形式: 0.134424
	プログラミング: 0.123071
	無分別: 0.119383
	西欧: 0.119383
440 PythonでTCPスタックを記述するとどうなる?
	パケット: 0.751933
	シーケンス: 0.276038
	番号: 0.249894
	応答: 0.207695
	送る: 0.141868
	送信: 0.119046
	消える: 0.112382
	スタック: 0.107581
	ステップ: 0.106178
	交信: 0.092077
269 リーダブル・コードを書く
	ハルク: 0.42382

	新入り: 0.264796
	文化: 0.217014
	社員: 0.214831
	規約: 0.193390
	チーム: 0.189800
	エンジニアリング: 0.183343
	品質: 0.176092
	社内: 0.175156
	ツール: 0.164873
319 Go言語がダメな理由
	構造: 0.300471
	ヒープ: 0.248796
	組み込み: 0.230641
	汎用: 0.228130
	言語: 0.225098
	関数: 0.210004
	プログラミング: 0.203883
	不変: 0.171098
	返す: 0.153569
	システム: 0.150282
770 モダンなWebプロジェクトにおけるベストプラクティス
	環境: 0.511349
	ファイル: 0.274799
	アプリケーション: 0.250903
	本番: 0.244449
	プロジェクト: 0.219216
	配信: 0.149230
	メンバー: 0.141074
	設定: 0.132230
	ブランチ: 0.130441
	ステージング: 0.121403
579 Heroku元CTOによる『ベルリンのスタートアップシーンへの旅』
	スタート: 0.360624
	ベルリン: 0.253286
	アップ: 0.248348
	ヨーロッパ: 0.220796
	企業: 0.212495
	サンフランシスコ: 0.169495
	ジェームズ: 0.151322
	起業: 0.128474
	スタッフ: 0.123757
	関わる: 0.118873
750 Servoにおける単位系の静的チェック
	単位: 0.692611
	画素: 0.259825
	座標: 0.248680
	変換: 0.186362
	バグ: 0.139931
	ライブラリー: 0.138816
	原点: 0.138816
	静的: 0.131293
	たとえば: 0.127779
	幾何: 0.111307
586 VagrantとDocker: Mac OS X上でPostgres、Elasticsearch、Redisをセットアップする方法
	キシ: 0.510677
	コンテナ: 0.501461
	プロ: 0.362659
	仮想: 0.251091
	マシン: 0.14728

### TF-IDFを用いた類似文書検索　コサイン類似度

In [21]:
# 文書集合を元にして各文書中の単語のTF-IDF値を計算
vectorizer = TfidfVectorizer(analyzer='word', max_df=0.5)
vecs = vectorizer.fit_transform(data)
# 類似度の計算
sim = cosine_similarity(vecs)
docs = zip(ids, sim[2])
# コサイン類似度が高い順にソートしてループ
for id, similarity in sorted(docs, key=lambda x: x[1], reverse=True):
    index = [k for k, v in enumerate(find_all_data) if v[columns.index('id')] == id][0]
    entry_title = find_all_data[index][columns.index('entry_title')]
    print(id, entry_title, similarity)

178 データサイエンティストにおいて、最も需要のあるスキルとは 1.0000000000000002
131 データサイエンス・ベン図 0.31670239769403524
689 データサイエンティストのためのEmacs 0.2565068205367657
674 機械学習とは何か？ – 機械学習の定義と使える言い回し 0.24202258982900401
78 機械学習のレベル別学習法 0.22025288233231022
82 機械学習に挑んだ一年間 – 機械学習について一から学び、仕事に活用するまでの道のり 0.21421632402573124
365 Stack Overflowのデータサイエンティストとしての1年間 0.20480607799580822
671 PythonとRuby：ワークシェイプの徹底比較！ 0.20461083455515786
664 機械学習はどの分野に属するのか？ 0.20086282336297684
573 私を魅了した起業家について 0.19134699186530324
585 アカデミアからデータサイエンティストへの転職で、私の時間の使い方がどう変わったか 0.1869927811305414
309 コーディングを学ぶこと、それはあなたが考えるよりも大変です 0.1580704789373656
60 日常にある機械学習の応用例 0.1564934686628205
101 機械学習の用語集：データ、学習とモデリング 0.15549478557957952
702 マネージャのためのAngularJSの概要 Part 1 0.14407727796709963
753 Haskellのエンジニアは二流なのか？（答えはノーである） 0.14275776463981005
121 優秀なJavaScriptの開発者になるための5か条 0.14180117690958993
813 機械学習アルゴリズムへの招待 0.1375109655033694
308 Amazon Redshiftで顧客分析ソリューションを構築 0.1364613000336475
340 なぜ採用される言語とされない言語があるのか 0.13488022205809386
608 効率的な統計実践のための、10個のシンプルなルール 0

487 @extendを使うべき時、@mixinを使うべき時 0.02975928784429964
301 Vimの生産性を高める12の方法 0.029319476011791742
311 私がどのようにしてキーロガーをクラックし、最終的に第三者の受信トレイに行きついたか 0.029262583698658574
16 パーリンノイズを理解する 0.0290142184265676
747 情報理論を視覚的に理解する (1/4) : 0.028962252751267223
328 『DOOM』（2016）-グラフィックス研究 – 前編 0.028946433685663256
421 ページネーションのベストプラクティス 0.028925659412792036
76 Q. （関数型）リアクティブプログラミングとは何ですか？ 0.028898213304687163
286 ES6 チートシート 0.028883879064058798
558 JavaScriptにおける継承のパターン4種類の概要と対比 0.028862757372720576
698 APIデザインにおける七つの大厄介 0.02875022981936987
10 Go言語の低レイテンシGC実現のための取り組み 0.028698907550063813
73 なぜsystemdなのか？ 0.0286933950072038
687 Dockerの本番運用 0.02867596430691523
367 強固なデータ・インフラストラクチャを構築するためのログの活用（デュアル書き込みがダメな理由）PART 2 0.028668041285854432
466 画像処理入門講座 : OpenCVとPythonで始める画像処理 0.02863727948169468
262 Pythonを教えるためのいくつかの提案 0.028624079569063536
730 Haskell、OCaml、RacketでGCのレイテンシを測る 0.028445325292077994
604 WebSocket大合戦：Clojure、C++、Elixir、Go、NodeJS、Ruby 0.028385815927448096
230 CSS will-changeプロパティについて知っておくべきこと 0.028

### 類似文書検索 同一タグが含まれているもののみ

In [24]:
# 文書集合を元にして各文書中の単語のTF-IDF値を計算
vectorizer = TfidfVectorizer(analyzer='word', max_df=0.9)
vecs = vectorizer.fit_transform(data)
# 類似度の計算
sim = cosine_similarity(vecs)
for i in range(0, len(sim)):
    target_id = find_all_data[i][columns.index('id')]
    target_tag = find_all_data[i][columns.index('tag')]
    if target_tag is None:
        continue
    target_tag_set = set(target_tag.split(','))
    docs = zip(ids, sim[i])
    count = 0
    similaritys = []
    # コサイン類似度が高い順にソートしてループ
    for id, similarity in sorted(docs, key=lambda x: x[1], reverse=True):
        if target_id == id:
            continue
        
        index = [k for k, v in enumerate(find_all_data) if v[columns.index('id')] == id][0]
        entry_title = find_all_data[index][columns.index('entry_title')]
        tag = find_all_data[index][columns.index('tag')]
        # 同一タグが含まれるかの判定
        if tag is None:
            continue
        tag_set = set(tag.split(','))
        if len(set(target_tag_set) - set(tag_set)) != 0:
            continue
        
        similaritys.append({'id': id, 'similarity': similarity})
        if count >= 4:
            break
        count += 1
    # コサイン類似度が高い記事IDを最大5件登録
    try:
        if(len(similaritys) > 0):
            mysql_class.set_article_by_id('similarity', target_id, similaritys)
    except:
        import traceback
        traceback.print_exc()
        mysql_class.disconnect()


### 対象の記事と類似する記事を出力

In [25]:
# 検索タイトル
target_title = 'PHPはもうダメだ、PHP万歳！'
article = mysql_class.find_article_by_entry_title(columns, target_title)
if (article is not None):
    print('--- 関連タイトル ---')
    for similarity in json.loads(article[columns.index('similarity')]):
        similarity_article = mysql_class.find_article_by_id(columns, similarity['id'])
        print(similarity_article[columns.index('entry_title')])

--- 関連タイトル ---
ある中級プログラマの告白
HHVMの今後
セカンドシステム症候群にまつわる3つの話―Perl 6, Python 3, PHP 6
高速なWebサーバアプリケーションを構築するための6つの経験則
最も割高なアンチパターン : 構造化されたデータを文字列関数で操作する「printfアンチパターン」について
